In [13]:
# main.py
from fastapi import FastAPI, Request, Query, Path, HTTPException, status
from pydantic import BaseModel
from datetime import date, datetime, timedelta
from typing import List, Optional
from fastapi.responses import JSONResponse
import json

from graph_builder import build_graph
from tools import query_db_mcp
from config import config
import uvicorn
import random
import asyncio

from fastapi.middleware.cors import CORSMiddleware

# services/llm_service.py에서 새로운 에이전트 함수 임포트
from services.llm_service import agent_chat_with_claude, generate_and_update_chat_title

In [14]:
# /chat/{chat_id}/message 요청 모델
class NewMessage(BaseModel): #! 수정 필요 - role 추가, 이 role은 User 아니면 AI.
    msg: str

In [29]:
new_message = NewMessage
new_message.msg = "감사합니다."
chat_id = 8106

In [ ]:
chat_history = []
try:
    #? 1. DB에서 전체 대화 기록 조회 (새 스키마에 맞게 수정)
    history_query = f"SELECT message FROM chat_messages WHERE chat_id = {chat_id} ORDER BY created_at ASC"
    db_result = await query_db_mcp(db_id=config.GLUCOBEAT_DB_ID, query=history_query, query_type="query")
    
    for row in db_result.get("rows"):
        if row.get("message"):
            message_text = json.loads(row["message"])
            chat_history.append(message_text)
            
except Exception as e:
    print(f"WARN: DB에서 채팅 기록 조회 실패: {e}")
    chat_history = []

In [39]:
try:
    #? 2. 사용자 메시지를 DB에 INSERT (간단한 JSON 구조로 저장)
    # 이때 이러한 사용자 메시지는 아래처럼 저장해야. 
    import json
    user_message = {"role": "user", "content": new_message.msg}
    message_json = json.dumps(user_message, ensure_ascii=False).replace("'", "''")
    user_sql = f"INSERT INTO chat_messages (chat_id, message) VALUES ({chat_id}, '{message_json}')"
    await query_db_mcp(db_id=config.GLUCOBEAT_DB_ID, query=user_sql, query_type="query")
except Exception as e:
    print(f"ERROR: 사용자 메시지 DB 저장 실패: {e}")

In [40]:
print(user_sql)

INSERT INTO chat_messages (chat_id, message) VALUES (8106, '{"role": "user", "content": "감사합니다."}')


In [41]:
#? 3. LLM 호출을 위해 현재 메시지를 대화 기록에 추가
# 이는 아래처럼 추가해야 하는 수 밖에.
chat_history.append(user_message)

print(f"DEBUG: Final chat_history before LLM call: {chat_history}")  # 디버깅 로그

DEBUG: Final chat_history before LLM call: [{'role': 'user', 'content': '혈당 수치를 정리해 주세요.'}, {'role': 'assistant', 'content': '혈당 수치를 정리해드리겠습니다. 먼저 데이터베이스에서 혈당 기록을 조회하겠습니다.'}, {'role': 'tool', 'content': "{'columns': ['time', 'id', 'cgm_value'], 'rows': [{'time': '2025-06-29T18:00:00', 'id': 1, 'cgm_value': 130.0}, {'time': '2025-06-29T12:00:00', 'id': 1, 'cgm_value': 125.0}, {'time': '2025-06-29T08:00:00', 'id': 1, 'cgm_value': 110.0}, {'time': '2025-06-25T12:00:00', 'id': 1, 'cgm_value': 150.0}]}"}, {'role': 'assistant', 'content': '혈당 수치 정리:\n\n## 최근 혈당 기록 (CGM 수치)\n\n| 날짜 및 시간 | 혈당 수치 (mg/dL) |\n|-------------|------------------|\n| 2025-06-29 18:00 | 130.0 |\n| 2025-06-29 12:00 | 125.0 |\n| 2025-06-29 08:00 | 110.0 |\n| 2025-06-25 12:00 | 150.0 |\n\n## 혈당 수치 분석\n\n**최신 혈당 수치**: 130 mg/dL (2025-06-29 18:00)\n\n**혈당 범위**:\n- 최고: 150 mg/dL (2025-06-25)\n- 최저: 110 mg/dL (2025-06-29 아침)\n\n**혈당 수치 평가**:\n- 110 mg/dL: 정상 범위\n- 125-130 mg/dL: 약간 높음 (식후 또는 스트레스 상황)\n- 150 mg/dL: 높음 (주의 필요)

In [42]:
#? 4. 첫 번째 메시지인 경우 제목 생성과 LLM 응답을 병렬로 처리
if len(chat_history) == 1:
    # 첫 번째 사용자 메시지 - 제목 생성과 LLM 응답을 병렬 실행
    print(f"DEBUG: 첫 번째 메시지 감지 - 병렬 처리 시작")
    llm_task = agent_chat_with_claude(chat_history)
    title_task = generate_and_update_chat_title(chat_id, new_message.msg)
    
    # 병렬 실행 후 결과 수집
    llm_message_list, generated_title = await asyncio.gather(llm_task, title_task)
    print(f"DEBUG: 병렬 처리 완료 - 제목: '{generated_title}'")
    print(f"DEBUG: 병렬 처리 완료 - 응답: '{llm_message_list}'")
else:
    # 기존 메시지 - 일반 LLM 응답만 처리
    llm_message_list = await agent_chat_with_claude(chat_history)

In [43]:
llm_message_list

[{'role': 'user', 'content': '혈당 수치를 정리해 주세요.'},
 {'role': 'assistant',
  'content': '혈당 수치를 정리해드리겠습니다. 먼저 데이터베이스에서 혈당 기록을 조회하겠습니다.'},
 {'role': 'tool',
  'content': "{'columns': ['time', 'id', 'cgm_value'], 'rows': [{'time': '2025-06-29T18:00:00', 'id': 1, 'cgm_value': 130.0}, {'time': '2025-06-29T12:00:00', 'id': 1, 'cgm_value': 125.0}, {'time': '2025-06-29T08:00:00', 'id': 1, 'cgm_value': 110.0}, {'time': '2025-06-25T12:00:00', 'id': 1, 'cgm_value': 150.0}]}"},
 {'role': 'assistant',
  'content': '혈당 수치 정리:\n\n## 최근 혈당 기록 (CGM 수치)\n\n| 날짜 및 시간 | 혈당 수치 (mg/dL) |\n|-------------|------------------|\n| 2025-06-29 18:00 | 130.0 |\n| 2025-06-29 12:00 | 125.0 |\n| 2025-06-29 08:00 | 110.0 |\n| 2025-06-25 12:00 | 150.0 |\n\n## 혈당 수치 분석\n\n**최신 혈당 수치**: 130 mg/dL (2025-06-29 18:00)\n\n**혈당 범위**:\n- 최고: 150 mg/dL (2025-06-25)\n- 최저: 110 mg/dL (2025-06-29 아침)\n\n**혈당 수치 평가**:\n- 110 mg/dL: 정상 범위\n- 125-130 mg/dL: 약간 높음 (식후 또는 스트레스 상황)\n- 150 mg/dL: 높음 (주의 필요)\n\n일반적으로 건강한 성인의 혈당 수치는:\n- 공복 시:

In [44]:
#? 5. 받은 메시지 리스트에서 새로 추가된 부분만 DB에 저장
original_length = len(chat_history)
new_messages = llm_message_list[original_length:]  # 새로 추가된 AI/Tool 메시지들


In [45]:
new_messages

[{'role': 'assistant',
  'content': '천만에요! 혈당 수치 정리가 도움이 되었다니 다행입니다. \n\n혈당 관리는 꾸준한 모니터링이 중요하니, 언제든지 추가로 혈당 데이터 분석이나 건강 관련 정보가 필요하시면 말씀해 주세요. 건강하세요! 😊'}]

In [46]:
try:
    if new_messages:
        for message in new_messages:
            message_json_string = json.dumps(message, ensure_ascii=False)
            
            # 플레이스홀더를 '?'에서 '%s'로 수정
            llm_sql = "INSERT INTO chat_messages (chat_id, message) VALUES (%s, %s)"
            
            params_tuple = (chat_id, message_json_string)
            
            # query_type은 서버에서 특별히 처리하지 않으므로 "query"로 유지해도 무방합니다.
            await query_db_mcp(
                db_id=config.GLUCOBEAT_DB_ID,
                query=llm_sql,
                params=params_tuple,
                query_type="query" 
            )

except Exception as e:
    print(f"WARN: LLM 응답 DB 저장 실패: {e}")

---

In [1]:
import os
import asyncio
from typing import List, Dict, Any, AsyncGenerator
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

# 현재 프로젝트의 다른 모듈 임포트 (상대 경로 수정)
import sys
import os
from tools import query_db_mcp
from config import config


In [2]:

# .env 파일에서 환경 변수 불러오기
load_dotenv()


True

In [3]:

# --- 상수 정의 ---
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
CLAUDE_MODEL = "claude-sonnet-4-20250514" #! 채팅용 모델 - 변경 절대금지
CLAUDE_HAIKU_MODEL = "claude-3-5-haiku-latest"  #! 제목 생성용 경량 모델 - 변경 절대금지


In [4]:

# 동적 시스템 프롬프트 함수 (2025 베스트 프랙티스)
def get_system_prompt() -> str:
    """GlucoBeat 의료 어시스턴트용 시스템 프롬프트 반환"""
    return """
You are a helpful medical assistant named "GlucoBeat".
You need to talk to users in a friendly and professional manner.
Your primary role is to provide information and support related to blood sugar management and diabetes based on the user's data.
But, do not use emoji. You have to provide information in a friendly and professional manner, without emojis.

First, try to answer the user's question using the existing conversation history.
If the user asks for specific data that is not in the conversation (e.g., "what is my latest blood sugar?", "how much insulin did I take yesterday?"), then you should use the `query_database` tool to get the information.
You have access to a database with the following tables:
- `cgm_records`: Contains the user's CGM (Continuous Glucose Monitoring) data.
- `insulin_records`: Contains the user's insulin injection history.

IMPORTANT: All queries MUST be for the user with `id = 1`.
When generating SQL, use the following table schemas:
- `cgm_records(time, id, cgm_value)`
- `insulin_records(time, id, insulin_amount)`

You must not provide a medical diagnosis. Always recommend that the user consult a doctor for any medical advice.
"""


In [77]:
# --- LangChain 도구 정의 ---
@tool
async def query_database(query: str) -> str:
    """
    Use this tool to query the user's health database to answer questions.
    The input must be a valid SQL query for MySQL.
    For example, to get the latest CGM value, use:
    'SELECT cgm_value FROM cgm_records WHERE id = 1 ORDER BY time DESC LIMIT 1'
    For yesterday's data, use:
    'SELECT * FROM cgm_records WHERE id = 1 AND DATE(time) = DATE_SUB(CURDATE(), INTERVAL 1 DAY)'
    """
    # print(f"--- Executing Database Query via MCP ---")
    
    # MySQL 문법 호환성을 위한 쿼리 변환
    mysql_query = query.replace(
        "DATE('now', '-1 day')", 
        "DATE_SUB(CURDATE(), INTERVAL 1 DAY)"
    ).replace(
        "DATE('now')", 
        "CURDATE()"
    )
    
    # print(f"Original SQL: {query}")
    # if mysql_query != query:
    #     print(f"MySQL Compatible SQL: {mysql_query}")
    
    try:
        result = await query_db_mcp(db_id=config.GLUCOBEAT_DB_ID, query=mysql_query)
        # print(f"Result: {result}")
        # LLM이 결과를 쉽게 이해하도록 문자열로 변환
        return str(result)
    except Exception as e:
        # print(f"Error executing query: {e}")
        return f"Error: Could not retrieve data. {e}"


In [78]:
tools = [query_database]

llm = ChatAnthropic(model=CLAUDE_MODEL, api_key=ANTHROPIC_API_KEY)

In [79]:
app = create_react_agent(
    model=llm,
    tools=tools,
    prompt=get_system_prompt()
)

# --- 테스트용 대화 기록 ---
chat_history = [
    {"role": "user", "content": "3일 전 혈당 기록을 시간 순서대로 보여줘"}
]

In [80]:
async def agent_chat_with_claude_stream(chat_history: List[Dict[str, str]]):
    """
    채팅 기록을 받아서 AI 응답을 추가한 전체 대화 기록을 반환
    
    Args:
        chat_history: [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
    
    Yields:
        Server-Sent Events (SSE) 형식의 JSON 문자열입니다.
        이벤트 유형('text', 'tool_call', 'tool_result')에 따라 다른 데이터를 포함합니다.
        예: 'data: {"type": "text", "content": "..."}\n\n'
    """
    # 대화 기록을 LangChain 메시지 형식으로 변환
    messages = []
    for message in chat_history:
        role = message.get("role")
        content = message.get("content", "")
        
        if role == "user":
            messages.append(HumanMessage(content=content))
        elif role == "assistant":
            messages.append(AIMessage(content=content))

    # LangGraph Stateless 실행 (main.py chat_history 기반)
    inputs = {"messages": messages}
    
    # 에이전트 실행 및 결과 추출 (메모리 없는 순수 처리)
    async for event in app.astream_events(inputs, version='v2'):
        kind = event["event"]

        # 1. LLM이 생성하는 텍스트/도구 호출 스트리밍
        if kind == "on_chat_model_stream":
            # Anthropic 모델의 chunk.content는 항상 content block의 리스트입니다.
            for part in event["data"]["chunk"].content:
                # 텍스트 블록만 스트리밍합니다. (tool_use는 on_tool_start에서 처리)
                if part.get("type") == "text" and part.get("text"):
                    yield f"data: {json.dumps({'type': 'text', 'content': part['text']}, ensure_ascii=False)}\n\n"

        # ✨ 2. (수정) 도구 호출 시작 시점 포착!
        # on_chat_model_stream에서 tool_use를 처리하는 대신 on_tool_start를 사용합니다.
        elif kind == "on_tool_start":
            response_json = {
                "type": "tool_call",
                "content": {
                    "name": event["name"],
                    "args": event["data"].get("input") # 이 이벤트에는 모든 인자가 포함되어 있습니다.
                }
            }
            yield f"data: {json.dumps(response_json, ensure_ascii=False)}\n\n"

        # 3. 도구 실행 결과 스트리밍 (수정 없음)
        elif kind == "on_tool_end":
            response_json = {
                "type": "tool_result",
                "content": {
                    "name": event["name"],
                    "output": str(event["data"].get("output"))
                }
            }
            yield f"data: {json.dumps(response_json, ensure_ascii=False)}\n\n"


In [81]:
# 스트리밍 함수를 호출하고 결과를 실시간으로 출력
async for chunk in agent_chat_with_claude_stream([{"role":"user", "content" : "어제 혈당 수치를 정리해 주세요."}]):
    print(chunk, end="||", flush = True)
    
# 크게 세 가지로 구성 - text, tool_call, tool_result


data: {"type": "text", "content": "어제의"}

||data: {"type": "text", "content": " 혈당 수치를"}

||data: {"type": "text", "content": " 확인해드리겠습니다."}

||data: {"type": "tool_call", "content": {"name": "query_database", "args": {"query": "SELECT TIME(time) as measurement_time, cgm_value FROM cgm_records WHERE id = 1 AND DATE(time) = DATE_SUB(CURDATE(), INTERVAL 1 DAY) ORDER BY time"}}}

||data: {"type": "tool_result", "content": {"name": "query_database", "output": "content=\"{'columns': ['measurement_time', 'cgm_value'], 'rows': []}\" name='query_database' tool_call_id='toolu_01DnoV3i5iKhminNPKUy2hTw'"}}

||data: {"type": "text", "content": "어"}

||data: {"type": "text", "content": "제 기록된 혈당 수치가 없는 것 같습니다."}

||data: {"type": "text", "content": " 혹시 CGM 기기가 제대로 작동하고"}

||data: {"type": "text", "content": " 있는지 확인해보시겠어요?\n\n최"}

||data: {"type": "text", "content": "근 며칠간의 혈당 데이터가"}

||data: {"type": "text", "content": " 있는지 확인해보겠습니다."}

||data: {"type": "tool_call", "content": {"name": "query_da

In [82]:
async def agent_chat_with_claude(chat_history: List[Dict[str, str]]) -> List[Dict[str, str]]:
    """
    채팅 기록을 받아서 AI 응답을 추가한 전체 대화 기록을 반환
    
    Args:
        chat_history: [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
    
    Returns:
        chat_history + [{"role": "assistant", "content": "AI 응답"}]
    """
    if not chat_history:
        return [{"role": "assistant", "content": "대화 기록이 비어있습니다."}]

    try:
        # 대화 기록을 LangChain 메시지 형식으로 변환
        messages = []
        for message in chat_history:
            role = message.get("role")
            content = message.get("content", "")
            
            if role == "user":
                messages.append(HumanMessage(content=content))
            elif role == "assistant":
                messages.append(AIMessage(content=content))

        # LangGraph Stateless 실행 (main.py chat_history 기반)
        inputs = {"messages": messages}
        
        # 에이전트 실행 및 결과 추출 (메모리 없는 순수 처리)
        result = await app.ainvoke(inputs)
        final_messages = result.get("messages", [])
        
        updated_history = []
        
        for msg in final_messages:
            if isinstance(msg, AIMessage):
                # AIMessage content 처리 개선
                content = msg.content
                if isinstance(content, list):
                    # Claude의 복합 응답 (text + tool_use)
                    text_parts = [part.get('text', '') for part in content if isinstance(part, dict) and part.get('type') == 'text']
                    final_text = ' '.join(text_parts) if text_parts else str(content)
                    updated_history.append({"role": "assistant", "content": final_text})
                else:
                    # 단순 텍스트 응답
                    updated_history.append({"role": "assistant", "content": str(content)})
            elif isinstance(msg, ToolMessage):
                # Tool 메시지는 role을 "tool"로 설정하여 구분
                updated_history.append({"role": "tool", "content": str(msg.content)})
            elif hasattr(msg, 'content'):  # 기타 메시지 타입
                updated_history.append({"role": "system", "content": str(msg.content)})
        
        # 만약 응답이 없다면 기본 메시지 추가
        if not final_messages:
            updated_history.append({"role": "assistant", "content": "응답을 생성하지 못했습니다."})
        
        return updated_history
        
    except Exception as e:
        print(f"LangGraph execution error: {e}")
        # 에러 발생 시에도 기존 대화 기록에 에러 메시지 추가
        error_history = chat_history.copy()
        error_history.append({
            "role": "assistant", 
            "content": f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}"
        })
        return error_history


In [83]:
await agent_chat_with_claude([{"role":"user", "content" : "어제 혈당 수치를 정리해 주세요."}])

[{'role': 'system', 'content': '어제 혈당 수치를 정리해 주세요.'},
 {'role': 'assistant', 'content': '어제의 혈당 수치를 조회해서 정리해드리겠습니다.'},
 {'role': 'tool', 'content': "{'columns': ['time', 'cgm_value'], 'rows': []}"},
 {'role': 'assistant',
  'content': '어제의 혈당 데이터가 없는 것 같습니다. 혹시 다른 날짜의 데이터를 확인해보시겠습니까? \n\n최근 며칠간의 혈당 수치를 확인해보거나, 특정 날짜의 데이터를 원하시면 말씀해 주세요. 또는 현재 사용 가능한 혈당 기록이 있는지 확인해보겠습니다.'},
 {'role': 'tool',
  'content': "{'columns': ['date', 'record_count'], 'rows': [{'date': '2025-06-29', 'record_count': 3}, {'date': '2025-06-25', 'record_count': 1}]}"},
 {'role': 'assistant',
  'content': '데이터베이스를 확인한 결과, 어제(2025-06-29 기준으로 전날) 기록이 없고, 현재 다음 날짜들의 혈당 기록이 있습니다:\n\n- 2025년 6월 29일: 3개 기록\n- 2025년 6월 25일: 1개 기록\n\n이 중 어떤 날짜의 혈당 수치를 정리해드릴까요? 아니면 가장 최근 기록인 6월 29일의 데이터를 보여드릴까요?'}]